In [5]:
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import os

In [12]:
ckey = 'RMNErsbyNWgPP2vEpsSBG6Gr9'
consumer_secret = 'AfttMkzkROoiOgNn340QdjBu8WseuTOjyKPG0hbWFzSstULiCo'
access_token_key = '285008880-CXc7m5CranacCoLN5rqjS84WfAsuNOvT006rNpKa'
access_token_secret = 'qEIk8pzuHbt2plyx2xKMkezwX3zS3fY7HwKpc1m5GpYYa'
 
 
start_time = time.time() #grabs the system time

keyword_list = ['hacking team'] #track list


In [13]:
import tweepy
import sys
import pika
import json
import time
import pprint
pp = pprint.PrettyPrinter(indent=1)

#get your own twitter credentials at dev.twitter.com
consumer_key = ckey
consumer_secret = consumer_secret
access_token = access_token_key
access_token_secret = access_token_secret

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

class CustomStreamListener(tweepy.StreamListener):
    def __init__(self, api):
        self.api = api
        super(tweepy.StreamListener, self).__init__()

        #setup rabbitMQ Connection
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(host='localhost')
        )
        self.channel = connection.channel()

        #set max queue size
        args = {"x-max-length": 2000}

        self.channel.queue_declare(queue='twitter_topic_feed', arguments=args)

    def on_status(self, status):
        print status.text
        print 'Localización: ', status.geo
        print 'Source: ' , status.source
        print 'Created at:' , status.created_at
        print "Lang: ", status.lang
        
        print
        
        print

        data = {}
        data['text'] = status.text
        data['created_at'] = time.mktime(status.created_at.timetuple())
        data['geo'] = status.coordinates
        data['source'] = status.source
 
        
        #queue the tweet
        self.channel.basic_publish(exchange='',
                                    routing_key='twitter_topic_feed',
                                    body=json.dumps(data))

    def on_error(self, status_code):
        print 'Encountered error with status code:', status_code
        print >> sys.stderr, 'Encountered error with status code:', status_code
        return True  # Don't kill the stream

    def on_timeout(self):
        print >> sys.stderr, 'Timeout...'
        return True  # Don't kill the stream


In [14]:

sapi = tweepy.streaming.Stream(auth, CustomStreamListener(api))
# my keyword today is chelsea as the team just had a big win
sapi.filter(track=keyword_list)  

RT @_JohnLeonard: Hacking Team MD used 'Passw0rd' as password – for every system http://t.co/s6ACw4XJaB via @Computing_News
Localización:  None
Source:  Tweetbot for iΟS
Created at: 2015-07-06 11:40:26
Lang:  en


#HackingTeam hacked as tables turn on notorious surveillance company http://t.co/Bk9sDqrdXF #CyberSecurity
Localización:  None
Source:  Twitter Web Client
Created at: 2015-07-06 11:40:26
Lang:  en


Hacking Team piraté – 400 GB de données dans la nature http://t.co/1HlzDUQ1aU #geeks #feedly
Localización:  None
Source:  Twitter Web Client
Created at: 2015-07-06 11:40:29
Lang:  fr


Hacking Team hacked, attackers claim 400GB in dumped data http://t.co/ul7RK4v3CP via @csoonline
Localización:  None
Source:  Twitter Web Client
Created at: 2015-07-06 11:40:30
Lang:  en


RT @rubin: The Intercept publie les manuels des logiciels de surveillance de Hacking Team : https://t.co/plvxLZhjwq cc @sachaqs @parizot
Localización:  None
Source:  Twitter for Mac
Created at: 2015-07-06 11:40:31


KeyboardInterrupt: 